# Discrete Empirical Interpolation Method (DEIM)
This python notebook exposes and implements the Discrete Empirical Interpolation Method (DEIM) for POD-Galerkin nonlinear model order reduction.

## 1. Model

$$ \mathbf u \in \mathbb R^{\mathcal N}; \quad \mathbf f(\mathbf u) \in \mathbb R^{\mathcal N};$$
$$ \left\{ 
\begin{array}{rcl}
\mathbf A \cdot\frac{\mathrm d \, \mathbf u}{\mathrm d t}&=&\mathbf f(\mathbf u), \\
\mathbf u(t_0) & = & \mathbf u_0, \;\mathbf u(t_n) \equiv \mathbf u_n.
\end{array}
\right.$$

## 2. POD

### 2.1 Snapshots matrix
Snapshots matrix $ \mathbf U = \big[\mathbf u_n \big]_{1\leq n\leq N_T} $.


### 2.2 POD basis
Correlation matrix $ \mathbf C_{\mathbf U} = \mathbf U^{\intercal}\cdot \mathbf U $.

Eigen vectors and eigen values $\{\mathbf a_i, \,\lambda_i\}_{1\leq i \leq N_T}$ s.t. $\mathbf C_{\mathbf U}\cdot\mathbf a_i = \lambda_i\,\mathbf a_i $.

POD basis $\{\phi_i\}_{1\leq i \leq N_T}$ with $\phi_i = \mathbf U \cdot \mathbf a_i$.

Reduced basis $\{\phi_i\}_{1\leq i \leq M}$ with $M<N_T$.

Reduced basis matrix $\Phi = \big[ \phi_i \big]_{1\leq i \leq M} \in \mathbb R^{\mathcal N \times M}$ 

### 2.3. Reduced order model
Projection of the solution on $\mbox{span}\{\Phi\}$:
$$\mathbf u(t) \simeq \Phi\cdot\mathbf c(t). $$

The POD-Galerkin reduced order model is given by
$$\Phi^\intercal \cdot \mathbf A \cdot \Phi \frac{\mathrm d \, \mathbf c}{\mathrm d t} = \Phi^\intercal \cdot \mathbf f(\Phi \cdot \mathbf c).  $$

## 3. DEIM

### 3.1 POD basis for the nonlinear term
Construct the snapshots for the nonlinear term
$$ \mathbf F = \big[\mathbf f(\mathbf u_n) \big]_{1\leq n \leq N_T} \in \mathbb R^{\mathcal N \times N_T}, $$
and construct the basis with the POD procedure described in section 2:
* Correlation matrix $ \mathbf C_{\mathbf F} = \mathbf F^{\intercal}\cdot \mathbf F $.
* Eigen vectors and eigen values $\{\mathbf b_i, \,\gamma_i\}_{1\leq i \leq N_T}$ s.t. $\mathbf C_{\mathbf F}\cdot\mathbf b_i = \gamma_i\,\mathbf b_i $.
* POD basis $\{\psi_i\}_{1\leq i \leq N_T}$ with $\psi_i = \mathbf F \cdot \mathbf b_i \in \mathbb R^{\mathcal N}$.
* Reduced basis $\{\psi_i\}_{1\leq i \leq M}$ with $M<N_T$.
* Reduced basis matrix $\Psi = \big[ \psi_i \big]_{1\leq i \leq M} \in \mathbb R^{\mathcal N \times M}$ 



### 3.2 Projector
Define the projector
$$ \mathbf P = [\mathbf I_{p_k}]_{1\leq k \leq M} \in \mathbb R^{\mathcal N \times M} $$
where $\mathbf I_k\in\mathbb R^{\mathcal N}$ denotes the $k$-th column of the identity matrix, and $[p_k]_{1, \leq k \leq M}$ is a set of indices chosen so that $$p_k = \underset{i\in[0\cdots\mathcal N]}{\mbox{argmax}}\Big[\left\vert\psi_k - \tilde \psi_{k} \right\vert \big]_i,$$
$$\begin{array}{rcl}
\tilde \psi_{k}  &=& \Psi_{k-1}\cdot\big(\mathbf P^\intercal_{k-1} \cdot \Psi_{k-1} \big)^{-1}\cdot\mathbf P^\intercal_{k-1} \cdot \Psi_{k-1}, \\ 
\mathbf P_{k} &=& [\mathbf P_i]_{1\leq i \leq k} \in \mathbb R^{\mathcal N \times K},\\
\Psi_{K} &=& [\Psi_i]_{1\leq i \leq K} \in \mathbb R^{\mathcal N \times K}.\end{array}$$

### 3.3 Reduced order model
Projection of the nonlinear term on $\mbox{span}\{\Psi\}$:
$$\mathbf f\big(\mathbf u(t)\big) \simeq \Psi\cdot\big(\mathbf P^\intercal \cdot \Psi \big)^{-1}\cdot\mathbf P^\intercal \cdot \mathbf f \big(\mathbf u(t)\big). $$

In the case of $\mathbf f(\mathbf u) \equiv \left[ \begin{array}{c}f_1(u_1)\\ \vdots \\ f_{\mathcal N}(u_{\mathcal N}) \end{array} \right]$, the following holds
$$\mathbf P^\intercal \cdot \mathbf f \big(\mathbf u(t)\big) \simeq \tilde{\mathbf f} \big(\mathbf c(t)\big) $$
with $\mathbf u \simeq \Phi\cdot \mathbf c$ and $\tilde{\mathbf f}: \mathbb R^{M}\rightarrow \mathbb R^M$ given by
$$
\Big[\tilde{\mathbf f}(\mathbf c)\Big]_k = f_{p_k} \Big(\mathbf I_{p_k}^\intercal \cdot \Phi\cdot \mathbf c \Big)
$$


The DEIM-POD-Galerkin reduced order model is given by
$$\widehat{\mathbf A} \cdot \frac{\mathrm d \, \mathbf c}{\mathrm d t} = \widehat{\mathbf B}\cdot \tilde{\mathbf f} \big(\mathbf c\big),  $$
with
$$
\begin{array}{rcl}
\widehat{\mathbf A} &=& \Phi^\intercal \cdot \mathbf A \cdot \Phi,\\
\widehat{\mathbf B} &=& \Phi^\intercal \cdot \Psi\cdot\big(\mathbf P^\intercal \cdot \Psi \big)^{-1}. 
\end{array}
$$